# 💬 AgRAG

Este notebook configura um pipeline completo de perguntas e respostas (Q&A) com base em um arquivo PDF, usando RAG (Retrieval-Augmented Generation), FAISS, embeddings com HuggingFace, e LLM Gemma via Ollama.

In [2]:
import os
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.llms import OllamaLLM
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import PromptTemplate

In [5]:
CROP = "milho"

In [9]:
def load_pdf_text(crop):
    all_text = ""
    with pdfplumber.open(os.path.join("data", f"{crop}.pdf")) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"
    if not all_text.strip():
        raise ValueError(f"No text found in the PDF for crop: {crop}")
    return all_text

In [10]:
pdf_text = load_pdf_text(CROP)

In [11]:
# 📌 Quebra de texto e geração dos embeddings
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
docs = splitter.create_documents([pdf_text])

embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
db = FAISS.from_documents(docs, embedding)

In [12]:
# 📌 Configuração do modelo Gemma via Ollama
llm = OllamaLLM(model="gemma3:1b")  # Requer que o modelo esteja instalado via `ollama pull gemma3:1b`

In [13]:
# 📌 Criação do chain de perguntas e respostas (RAG)
template = """
        Você é um assistente agrícola especializado na cultura de {crop}. 
        Responda a pergunta abaixo em português do Brasil, com base exclusivamente no conteúdo fornecido.

        Contexto:
        {context}

        Pergunta: {input}
        Resposta:
        """

prompt = PromptTemplate(template=template, input_variables=["input", "crop"])

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

retriever = db.as_retriever(search_kwargs={"k": 10})

qa_chain = create_retrieval_chain(
    retriever,
    document_chain,
)

In [14]:
# 📌 Fazer perguntas ao modelo
pergunta = "O que é Bipolaris no Milho?"
context_vars = {
    "input": pergunta,
    "crop": CROP,
}

resposta = qa_chain.invoke(context_vars)
print("\n", resposta["answer"])


 Em milho, Bipolaris é um patógeno fúngico que pode causar diversas doenças, afetando a qualidade do grão e a produtividade. As doenças causadas por Bipolaris são diversas, com diferentes graus de gravidade. A principal característica do Bipolaris é o desenvolvimento de manchas ou lesões em diferentes partes do grão, especialmente na parte inferior e na área de contato com a testa. Essas lesões podem variar em tamanho, cor e forma, tornando o grão irregular e comprometendo sua qualidade.

**Tipos de Doenças Causadas por Bipolaris:**

*   **Danos da Parte Inferior (da "parte brava"):** É a forma mais comum de dano. As lesões podem ser brancas, escuras ou em forma de "X", e podem causar um amarelamento da área.
*   **Danos na Área de Contato com a Testa:**  As lesões nessa área podem ser de diferentes tamanhos e formas, e costumam ser mais acentuadas do que as doentes na parte inferior.
*   **Danos na Parte Superior (da "parte branca"):**  As lesões são geralmente mais leves e podem ser